In [20]:
# Importing libraries
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
from mpi4py import MPI
from ufl import (
    inner,
    grad,
    conj,
    TrialFunction,
    TestFunction,
    as_matrix,
    as_vector,
    dot,
    Or,
    Measure,
    conditional,
    SpatialCoordinate,
    dx
)
from scipy.interpolate import CubicSpline
from dolfinx.mesh import locate_entities_boundary, create_interval, locate_entities, create_submesh
from dolfinx.fem import FunctionSpace, Function, functionspace, Function, dirichletbc, locate_dofs_topological, petsc, form, assemble_matrix, Expression, assemble_scalar
import basix
from petsc4py.PETSc import ScalarType, KSP
from slepc4py.SLEPc import EPS  
colors_list = ["blue", "green", "red", "magenta", "black", "purple", "orange", "brown", "pink"]


In [ ]:
# Define element size for the mesh refinement
h_i = [ 0.0625 / 2,  #N=2
        0.0625 / 4,  #N=3
        0.0625 / 8,] #N=4


# Domain configuration
Lx = 1. # fluid domain length (without PML)
l_inner=Lx/2

# Physical configuration
rho_media_1 = 1. # kg/m^3 
vel_media_1 = 2.   # m/s
rho_media_2 = 1. # kg/m^3 
vel_media_2 = 1.   # m/s
# PML configuration
pml_thickness = Lx/4

# Solver configuration

requested_eigenvalues_number = 50 # Number of eigenvalues to compute

# Define the density and velocity functions  
def rho_function(x):    
    return np.where(x[0] > l_inner, rho_media_2,rho_media_1)

data_sound_speed = [
    1519.85, 1519.9, 1520, 1520.5, 1521.0, 1522, 1522.5, 1523, 1524, 1526,
    1527, 1528, 1530, 1532, 1534, 1535, 1536, 1537.5, 1538, 1538.5,
    1539, 1539, 1539, 1539,
]  # Oman
data_depth = [-200,-190, -180, -150, -145, -140, -135, -130, -125, -110, -95, -75, -60, -54, -49, -39, -25, -22.5, -21, -18, -15, -10, -8.5, -7, -6.5, -0]
depth = [-1 * d for d in reversed(data_depth)]
sound_speed = [d for d in reversed(data_sound_speed)]    
def vel_function(x):
    # print(len(sound_speed), len(depth))
    spline = CubicSpline(depth, sound_speed)

    return spline(x[0])  # Solo usa la primera componente
# Crear puntos finos para debuxo suave
depth_fine = np.linspace(depth[0], depth[-1], 500)    # mesma escala de profundidade
sound_speed_fine = vel_function(depth_fine)

# Debuxo
plt.figure(figsize=(6,4))
plt.plot(sound_speed_fine, depth_fine, label="Spline", color="blue")
plt.plot(sound_speed, depth, "o", label="Datos", color="orange")

plt.gca().invert_yaxis()  # profundidade aumenta cara abaixo
plt.xlabel("Velocidade [m/s]")
plt.ylabel("Profundidade [m]")
plt.title("Perfil de Velocidade vs Profundidade (Spline Oman)")
plt.grid(True)
plt.legend()
plt.show()